In [1]:
import os
import pandas as pd
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import neighbors
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import nltk
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")


from nltk.corpus import stopwords
import multiprocessing

cores = multiprocessing.cpu_count()
doc2VecModel = "doc2Vec_model.sav"
LRdoc2VecModel = "LR_doc2Vec_model.sav"

path1 = "../input/ps-dataset/powershell_benign_dataset/"
path2 = "../input/ps-dataset/malicious_pure"
#path3 = r"D:\Test Scripts"
labels = []
text = []
testText = []
newText = []
#new_str = re.sub('[^a-zA-Z0-9\n\.]', ' ', string)
fName= []
regex = r':\s*\n+'
vectorizer = CountVectorizer(stop_words='english', max_features=5000)

def test_bowser(path):
    for filename in os.listdir(path):
        fName.append(filename)
        filename = os.path.join(path, filename)
        with open(filename, encoding='utf-8') as f:
            t_content = f.read()
            t_content = t_content.lower()
            t_content = t_content.replace("(", "")
            t_content = t_content.replace(")", "")
            t_content = t_content.replace("[", "")
            t_content = t_content.replace("]", "")
            t_content = t_content.replace("$", "")
            t_content = t_content.replace("-", "")
            t_content = t_content.replace("=", "")
            t_content = t_content.replace("", "")
            t_content = t_content.replace("{", "")
            t_content = t_content.replace("}", "")
            t_content = t_content.replace(".", "")
            t_content = t_content.replace("_", "")
            t_content = t_content.replace("|", "")
            t_content = t_content.replace(":", "")
            t_content = t_content.replace("|", "")
            t_content = t_content.replace("/", "")
            t_content = t_content.replace("@", "")
            t_content = t_content.replace("?", "")
            t_content = t_content.replace(",", "")
            t_content = t_content.replace(">", "")
            t_content = t_content.replace("<", "")
            t_content = t_content.replace(";", "")
            t_content = t_content.replace("\\", "")
            t_content = t_content.replace("+", "")
            t_content = t_content.replace("\"","")
            t_content = t_content.replace("\'","")
            t_content = t_content.replace("`", "")
            t_content = t_content.replace("*", "")
            t_content = " ".join(t_content.split())
            newText.append(t_content)

def bowser(path, label):
    for filename in os.listdir(path):
        labels.append(label)
        filename = os.path.join(path, filename)
        with open(filename, encoding='utf-8') as f:
                    content = f.read()
                    content = content.lower()
                    content = content.replace("(", "")
                    content = content.replace(")", "")
                    content = content.replace("[", "")
                    content = content.replace("]", "")
                    content = content.replace("$", "")
                    content = content.replace("-", "")
                    content = content.replace("=", "")
                    content = content.replace("", "")
                    content = content.replace("{", "")
                    content = content.replace("}", "")
                    content = content.replace(".", "")
                    content = content.replace("_", "")
                    content = content.replace("|", "")
                    content = content.replace(":", "")
                    content = content.replace("|", "")
                    content = content.replace("/", "")
                    content = content.replace("@", "")
                    content = content.replace("?", "")
                    content = content.replace(",", "")
                    content = content.replace(">", "")
                    content = content.replace("<", "")
                    content = content.replace(";", "")
                    content = content.replace("\\", "")
                    content = content.replace("+", "")
                    content = content.replace("\"", "")
                    content = content.replace("\'", "")
                    content = content.replace("`", "")
                    content = content.replace("*", "")
                    content = " ".join(content.split())
                    testText.append(content)


#test_bowser(path3)
bowser(path1,"Benign")
bowser(path2,"Malicious")
#print(testText[6])
numFiles = len(testText)

df = pd.DataFrame(testText, index=range(numFiles), columns=["Powershell Code"])
df['Labels'] = labels
#df.to_csv('tester.csv')
print(df)

train, test = train_test_split(df, test_size=0.3, random_state=42)

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['Powershell Code']), tags=[r.Labels]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['Powershell Code']), tags=[r.Labels]), axis=1)

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

#Saving doc2vec model
pickle.dump(model_dbow, open(doc2VecModel, 'wb'))


y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

# logreg = LogisticRegression(n_jobs=1, C=1e5)
# logreg.fit(X_train, y_train)

# #Saving LR model
# pickle.dump(logreg, open(LRdoc2VecModel, 'wb'))

# y_pred = logreg.predict(X_test)

# print(classification_report(y_test,y_pred))

# #print(classification)
# #print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
# #print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))


In [2]:
import pickle

doc2VecModel = "./doc2Vec_model.sav"

model_dbow = pickle.load(open(doc2VecModel, 'rb'))

#Logreg on DBOW Doc2Vec
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)

#Saving LR model

y_pred = logreg.predict(X_test)

print(classification_report(y_test,y_pred,digits=6))

In [3]:
#KNN on DBOW Doc2Vec
KNN= KNeighborsClassifier(n_neighbors=5,weights='uniform',n_jobs=1,algorithm="auto",leaf_size=30)

KNN.fit(X_train,y_train)
pred = (KNN.predict(X_test))

print(classification_report(y_test,pred,digits=6))



In [55]:
import pickle
from sklearn.ensemble import RandomForestClassifier



#RF on DBOW Doc2Vec
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)


RF = RandomForestClassifier()
RF.fit(X_train,y_train)

y_pred = RF.predict(X_test)
print(classification_report(y_test,y_pred,digits=6))


In [4]:
# NB on DBOW Doc2Vec
NB = GaussianNB()
NB.fit(X_train,y_train)

pred = (NB.predict(X_test))

print(classification_report(y_test,pred,digits=6))


In [5]:
#Distributed Memory Model
model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(train_tagged.values)])

In [6]:
for epoch in range(30):
    model_dmm.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dmm.alpha -= 0.002
    model_dmm.min_alpha = model_dmm.alpha

In [49]:
#Distributed Memory Model
model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(train_tagged.values)])


for epoch in range(30):
    model_dmm.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dmm.alpha -= 0.002
    model_dmm.min_alpha = model_dmm.alpha

y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm, test_tagged)
    
KNN= KNeighborsClassifier(n_neighbors=5,weights='uniform',n_jobs=1,algorithm="auto",leaf_size=30)
KNN.fit(X_train, y_train)
y_pred = KNN.predict(X_test)

print(classification_report(y_test,y_pred,digits=6))


In [50]:
y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm, test_tagged)
NB = GaussianNB()
NB.fit(X_train, y_train)
y_pred = NB.predict(X_test)

print(classification_report(y_test,y_pred,digits=6))


In [51]:
y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print(classification_report(y_test,y_pred,digits=6))


In [52]:
from sklearn.svm import SVC

#SVM on DBOW Doc2Vec
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)


svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

y_pred = svclassifier.predict(X_test)
print(classification_report(y_test,y_pred,digits=6))


In [53]:
#RF with DMM
y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm, test_tagged)


RF = RandomForestClassifier()
RF.fit(X_train,y_train)

y_pred = RF.predict(X_test)
print(classification_report(y_test,y_pred,digits=6))


In [54]:
#SVM with DMM
y_train, X_train = vec_for_learning(model_dmm, train_tagged)
y_test, X_test = vec_for_learning(model_dmm, test_tagged)

svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

y_pred = svclassifier.predict(X_test)
print(classification_report(y_test,y_pred,digits=6))


## --------------------Model 2------------------------

In [14]:
dataset_2 = pd.read_csv('../input/ps-dataset-2/Dataset_2.csv')
train2, test2 = train_test_split(dataset_2, test_size=0.3, random_state=42)

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

train_tagged2= train2.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['Powershell Code']), tags=[r.Labels]), axis=1)
test_tagged2= test2.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['Powershell Code']), tags=[r.Labels]), axis=1)

model_dbow2 = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow2.build_vocab([x for x in tqdm(train_tagged2.values)])

for epoch in range(30):
    model_dbow2.train(utils.shuffle([x for x in tqdm(train_tagged2.values)]), total_examples=len(train_tagged2.values), epochs=1)
    model_dbow2.alpha -= 0.002
    model_dbow2.min_alpha = model_dbow.alpha

def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

Y_train2, X_train2 = vec_for_learning(model_dbow2, train_tagged2)
Y_test2, X_test2 = vec_for_learning(model_dbow2, test_tagged2)
print(len(X_train2))


In [15]:
RF2 = RandomForestClassifier()
RF2.fit(X_train2,Y_train2)
print(len(Y_train2))

y_pred = RF2.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

In [16]:
import time

KNN2= KNeighborsClassifier(n_neighbors=5,weights='uniform',n_jobs=1,algorithm="auto",leaf_size=30)
KNN2.fit(X_train2,Y_train2)
st = time.time()
y_pred = KNN2.predict(X_test2)
end = time.time()
print(classification_report(Y_test2,y_pred,digits=6))
print(end-st)

In [17]:
LR2 = LogisticRegression(class_weight="balanced")
LR2.fit(X_train2,Y_train2)


y_pred = LR2.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

In [18]:
naive_bayes_classifier2 = GaussianNB()
naive_bayes_classifier2.fit(X_train2, Y_train2)
y_pred = naive_bayes_classifier2.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

In [19]:
from mlxtend.plotting import plot_learning_curves

svclassifier2 = SVC(kernel='linear')
svclassifier2.fit(X_train2, Y_train2)

y_pred = svclassifier2.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

In [20]:
#----------------------------DMM-----------------------------#

In [21]:
#Distributed Memory Model
model_dmm2 = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dmm2.build_vocab([x for x in tqdm(train_tagged2.values)])


for epoch in range(30):
    model_dmm2.train(utils.shuffle([x for x in tqdm(train_tagged2.values)]), total_examples=len(train_tagged2.values), epochs=1)
    model_dmm2.alpha -= 0.002
    model_dmm2.min_alpha = model_dmm.alpha

y_train2, X_train2 = vec_for_learning(model_dmm2, train_tagged2)
y_test2, X_test2 = vec_for_learning(model_dmm2, test_tagged2)
    

In [56]:
svclassifier2 = SVC(kernel='linear')
svclassifier2.fit(X_train2, Y_train2)

y_pred = svclassifier2.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

In [57]:
naive_bayes_classifier2 = GaussianNB()
naive_bayes_classifier2.fit(X_train2, Y_train2)
y_pred = naive_bayes_classifier2.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

In [58]:
LR2 = LogisticRegression(class_weight="balanced")
LR2.fit(X_train2,Y_train2)


y_pred = LR2.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

In [59]:
import time

KNN2= KNeighborsClassifier(n_neighbors=5,weights='uniform',n_jobs=1,algorithm="auto",leaf_size=30)
KNN2.fit(X_train2,Y_train2)
st = time.time()
y_pred = KNN2.predict(X_test2)
end = time.time()
print(classification_report(Y_test2,y_pred,digits=6))
print(end-st)

In [60]:
RF2 = RandomForestClassifier()
RF2.fit(X_train2,Y_train2)
print(len(Y_train2))

y_pred = RF2.predict(X_test2)
print(classification_report(Y_test2,y_pred,digits=6))

## CROSS VALIDATION MODEL 2 ON DATASET 1 WITH DMM ##

In [64]:
Y_train3, X_train3 = vec_for_learning(model_dmm2, train_tagged)
Y_test3, X_test3 = vec_for_learning(model_dmm2, test_tagged)

y_pred = RF2.predict(X_test3)
print(classification_report(Y_test3,y_pred,digits=6))

In [65]:
st = time.time()
y_pred = KNN2.predict(X_test3)
end = time.time()
print(classification_report(Y_test3,y_pred,digits=6))
print(end-st)

In [66]:
y_pred = LR2.predict(X_test3)
print(classification_report(Y_test3,y_pred,digits=6))

In [67]:
y_pred = naive_bayes_classifier2.predict(X_test3)
print(classification_report(Y_test3,y_pred,digits=6))

In [68]:
y_pred = svclassifier2.predict(X_test3)
print(classification_report(Y_test3,y_pred,digits=6))

## CROSS VALIDATION MODEL 1 ON DATASET 2 WITH DMM ##

In [72]:
Y_train4, X_train4 = vec_for_learning(model_dmm, train_tagged2)
Y_test4, X_test4 = vec_for_learning(model_dmm, test_tagged2)

y_pred = RF.predict(X_test4)
print(classification_report(Y_test4,y_pred,digits=6))

In [76]:
y_pred = svclassifier.predict(X_test4)
print(classification_report(Y_test4,y_pred,digits=6))

In [73]:
y_pred = NB.predict(X_test3)
print(classification_report(Y_test3,y_pred,digits=6))

In [74]:
y_pred = logreg.predict(X_test4)
print(classification_report(Y_test4,y_pred,digits=6))

In [75]:
st = time.time()
y_pred = KNN.predict(X_test4)
end = time.time()
print(classification_report(Y_test4,y_pred,digits=6))
print(end-st)

## CROSS VALIDATION MODEL 1 ON DATASET 2 WITH DBOW ##

In [37]:
Y_train5, X_train5 = vec_for_learning(model_dbow, train_tagged2)
Y_test5, X_test5 = vec_for_learning(model_dbow, test_tagged2)


y_pred = RF.predict(X_test5)
print(classification_report(Y_test5,y_pred,digits=6))

In [38]:
y_pred = logreg.predict(X_test5)
print(classification_report(Y_test5,y_pred,digits=6))

In [39]:
y_pred = NB.predict(X_test5)
print(classification_report(Y_test5,y_pred,digits=6))

In [71]:
y_pred = svclassifier.predict(X_test5)
print(classification_report(Y_test5,y_pred))

In [41]:
st = time.time()
y_pred = KNN.predict(X_test5)
end = time.time()
print(classification_report(Y_test5,y_pred,digits=6))
print(end-st)

## CROSS VALIDATION MODEL 2 ON DATASET 1 WITH DBOW ##

In [42]:
Y_train6, X_train6 = vec_for_learning(model_dbow2, train_tagged)
Y_test6, X_test6 = vec_for_learning(model_dbow2, test_tagged)

st = time.time()
y_pred = KNN.predict(X_test6)
end = time.time()
print(classification_report(Y_test6,y_pred,digits=6))
print(end-st)


In [43]:
y_pred = svclassifier.predict(X_test6)
print(classification_report(Y_test6,y_pred,digits=6))

In [44]:
y_pred = NB.predict(X_test6)
print(classification_report(Y_test6,y_pred,digits=6))

In [45]:
y_pred = logreg.predict(X_test6)
print(classification_report(Y_test6,y_pred,digits=6))

In [46]:
y_pred = RF.predict(X_test6)
print(classification_report(Y_test6,y_pred,digits=6))